In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd
import re
from sklearn.utils import resample
import json

In [ ]:
#API key
client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))

## Data sampling

In [ ]:
# load all remaining data (data not included in prompt selection or prompting experiment): text, label
data = pd.read_csv("")

### Generate confidence scores

In [ ]:
# prompt for confidence scores
topic_v3_prompt_confidence = """
Du er en gennemsnitlig dansk nyhedsforbruger. Du får en artikel, og skal tildele den et passende antal kategorier svarende til de emner den omhandler. Artiklen omhandler oftest flere emner, men en kan også kun omhandle ét emne. 
Her er nogle generelle principper, du skal følge: Du skal anvende generel verdensviden og artiklens kontekst, hvor det er nødvendigt. Et emne skal bidrage med at fremføre artiklens væsentligste pointer for at kategorien tildeles. Det er muligt at samtagge flere kategorier, så to kategorier tilsammen skaber det emne, artiklen omhandler.
Kategorier: ”Begivenhed”: En hændelse, der finder sted - særligt for at markere noget vigtigt og evt. tilbagevendende. Herunder mærkedag, personlig begivenhed, sportsbegivenhed og underholdningsbegivenhed. ”Bolig”: Privatpersoners hjem. Herunder køb og salg, renovering/indretning og udlejning. ”Erhverv”: Emner relateret til beskæftigelsesmæssige forhold, inkl. arbejdspladser. Herunder ansættelsesforhold, offentlig instans og privat virksomhed. ”Dyr”: Levende flercellede organismer, ekskl. planter og mennesker. ”Katastrofe”: Begivenheder, som har negative konsekvenser for en større eller mindre gruppe (mennesker). Herunder mindre ulykke og større katastrofe. ”Kendt”: Personer, som ofte er alment kendt blandt den brede befolkning, fx deltagere i tv-programmer, politikere, sportspersonligheder, toperhvervsfolk og kongelige. ”Konflikt og krig”: Sammenstød, uoverensstemmelser eller stridigheder mellem to eller flere parter - evt. med voldelige konsekvenser. Herunder terror og væbnet konflit. ”Kriminalitet”: Ulovligheder, der omhandler fx dramatiske, dødelige eller sindsoprivende begivenheder, hvori politiet evt. har været involveret. Herunder bandekriminalitet, bedrageri og personfarlig kriminalitet. ”Kultur”: Den levevis, som er resultat af menneskelig aktivitet, og forestillingsverden, herunder skikke, holdninger og traditioner, ved en bestemt befolkningsgruppe i en bestemt periode. Herunder byliv, kunst, museum og seværdighed, or rejse. ”Livsstil”: Den bevidst valgte levemåde, som (en) privatperson(er) har - ofte for at signalere en bestemt identitet. Herunder erotik, familieliv, fritid, krop og velvære, mad og drikke og partnerskab. ”Politik”: Samfundsrelevante problemstillinger, værdier o.lign., som udføres på baggrund af en bestemt ideologi. Herunder international politik og national politik. ”Samfund”: Forhold for mennesker, der typisk deler samme geografiske område og/eller er knyttet sammen på anden vis, fx gennem indbyrdes afhængighed og tilknytning til nationalstat. Herunder bæredygtighed og klima, værdier, religion og tendens. ”Sport”: (Fysisk) aktivitet, som udøves individuelt eller på hold i professionelle sammenhænge, og hvor der ofte indgår specifikke regler og evt. udstyr. Herunder cykling, håndbold, fodbold, ketcher- og batsport og motorsport. ”Sundhed”: Helbredsmæssige forhold. Herunder kosmetisk behandling og sygdom og behandling. Teknologi”: Beskrivelser og anvendelser af tekniske hjælpemidler til udførelse af opgaver. Herunder forbrugerelektronik, kunstig intelligens og software. ”Transportmiddel”: Førbare anordninger, som kan fragte levende væsener og/eller artefakter. Herunder bil, mindre transportmiddel, offentlig transport og større transportmiddel. ”Uddannelse”: Strukturerede undervisningsforløb. Herunder grundskole, ungdomsuddannelse og videregående uddannelse. "Underholdning”: Forhold med evne til at more/frembringe glæde hos modtageren. Herunder litteratur, film og tv, musik og lyd og reality. ”Vejr”: Meterologiske forhold, fx forudsigelse eller afrapportering, ofte omhandlende et bestemt geografisk område på et bestemt tidspunkt. "Videnskab”: Metodologisk forskning og undersøgelse af bestemte dele af virkeligheden. Herunder naturvidenskab og samfundsvidenskab og humaniora. ”Økonomi”: Produktion, fordeling og forbrug (hos både privatpersoner og på statsligt niveau) af varer og tjenester. Herunder makroøkonomi og mikroøkonomi. 
Giv også en confidence score med to decimaler fra 0.00 til 1.00, der repræsenterer hvor sikker du er i din vurdering af topics, hvor 0 er meget usikker og 1 er meget sikker.
Giv et præcist svar i json: {{topics: [”kategori”, ”kategori”, ”kategori” ...], "confidence": "score"}}.
"""

In [ ]:
# Function to call GPT-4o with few-shot prompts
def fewshot_topic_annotation(text, prompt, fewshot_dataset):
    # Sample few-shot examples from dataset without current article
    fewshot_dataset = fewshot_dataset[fewshot_dataset['text'] != text]
    fewshot_examples = fewshot_dataset.sample(3).to_dict(orient='records')
    # Create few-shot parrt of prompt
    fewshot_examples = "\n".join([f"Artikel: {example['text']}. Artiklen omhandler dette/disse emne(r): {example['label']}." for example in fewshot_examples])

    try:
        # Make a request to GPT-4o
        response = client.chat.completions.create(
            model="gpt-4o-2024-08-06",
            messages=[
                {
                    "role": "system", 
                    "content": f"{prompt}. Her er tre eksempler på artikler og deres emner: {fewshot_examples}"
                },
                {
                    "role": "user", 
                    "content": f"Artikel: {text} \nArtiklen omhandler dette/disse emne(r):"
                }
            ],
            temperature=0,
            response_format={
                "type": "json_schema",
                "json_schema": {
                    "name": "topic_schema",
                    "schema": {
                        "type": "object",
                        "properties": {
                            "topics": {
                                "description": "Topics of the article",
                                "type": "string"
                            },
                            "confidence": {
                                "description": "Confidence score for the topics from 0.00 to 1.00",
                                "type": "number",
                                "minimum": 0.0,
                                "maximum": 1.0
                            }
                        },
                        "additionalProperties": False
                    }
                }
            }
        )

        # Extract  response
        topics = response.choices[0].message.content
        return topics
    except Exception as e:
        print(f"Error: {e}")
        return None
    
# Apply function and add results to df
data["llm_annotation"] = data["text"].apply(lambda text: fewshot_topic_annotation(text, prompt=topic_v3_prompt_confidence, fewshot_dataset=data))

# Save the results to a new CSV file
data.to_csv("", index=False)

In [ ]:
# extract confidence scores
def extract_confidence(value):
    match = re.search(r'"confidence":\s*"?(\d+\.\d+)"?', str(value))
    if match:
        return int(float(match.group(1)))
    return None  # Return None if no match is found

data["confidence_scores"] = data["llm_annotation"].apply(extract_confidence)

In [ ]:
# format topics from strings to lists
def annotation_to_list(annotation):
    annotation_list = []
    
    topics = ["Begivenhed", "Bolig", "Erhverv", "Dyr", "Katastrofe", "Kendt", "Konflikt og krig", "Kriminalitet", "Kultur", "Livsstil",
          "Politik", "Samfund", "Sport", "Sundhed", "Teknologi", "Transportmiddel", "Uddannelse", "Underholdning", "Vejr", "Videnskab", "Økonomi"]

    # Regular expression to capture words inside single/double quotes or just words
    annotation_topics = [topic.lower() for topic in re.findall(r'["\']?([a-zA-ZæøåÆØÅ\s]+)["\']?', annotation)]
    
    # Iterate over the topics and check for case-insensitive matches in the extracted topics
    for topic in topics:
        if topic.lower() in annotation_topics:
            annotation_list.append(topic)
    
    return annotation_list
    
data["label"] = data["label"].apply(annotation_to_list)
data["llm_annotation"] = data["llm_annotation"].apply(annotation_to_list)

### Random sampling

In [ ]:
data = data.sample(n=300)

### Selective sampling

In [ ]:
# Define bins and labels for confidence intervals
bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
labels = ["0-0.1", "0.1-0.2", "0.2-0.3", "0.3-0.4", "0.4-0.5", "0.5-0.6", "0.6-0.7", "0.7-0.8", "0.8-0.9", "0.9-1.0"]
data["confidence_interval"] = pd.cut(data["confidence_scores"], bins=bins, labels=labels, include_lowest=True)

# Determine correctness of LLM annotations
def classify_correctness(row):
    human_set = set(row["label"])
    llm_set = set(row["llm_annotation"])
    if human_set == llm_set:
        return "correct"
    elif human_set & llm_set:
        return "partially_correct"
    else:
        return "incorrect"

data["is_correct"] = data.apply(classify_correctness, axis=1)

# Calculate proportions for each group based on available data
group_counts = data.groupby(["confidence_interval", "is_correct"]).size()
total_available = group_counts.sum()
group_proportions = group_counts / total_available

# Calculate target sample sizes based on the proportions
total_samples = 300
sample_sizes = (group_proportions * total_samples).round().astype(int)

# Initialize an empty DataFrame for the sampled data
sampled_df = pd.DataFrame()

# Sample examples from each group without replacement
for (interval, correct), size in sample_sizes.items():
    if size > 0:
        # Subset data for this group
        group_data = data[(data["confidence_interval"] == interval) & (data["is_correct"] == correct)]
        
        # Sample without replacement, up to the size of the group
        if len(group_data) >= size:
            sampled_group = group_data.sample(n=size, random_state=42, replace=False)
        else:
            # If not enough examples, sample all available examples
            sampled_group = group_data
            print(f"Warning: Not enough examples in group ({interval}, {correct}). Adding all {len(group_data)} examples.")
        
        sampled_df = pd.concat([sampled_df, sampled_group])

# Reset index for the sampled DataFrame
sampled_df.reset_index(drop=True, inplace=True)

# Save df to csv
sampled_df.to_csv("", index=False)

## Format fine-tuning data

In [ ]:
sampled_data = pd.read_csv("")

In [ ]:
# create fine-tuning dataset with best-performing prompt and with simplest prompt
topic_v3_prompt = """
Du er en gennemsnitlig dansk nyhedsforbruger. Du får en artikel, og skal tildele den et passende antal kategorier svarende til de emner den omhandler. Artiklen omhandler oftest flere emner, men den kan også omhandle kun ét emne.
Kategorier: ”Begivenhed”, ”Bolig”, ”Erhverv”, ”Dyr”, ”Katastrofe”, ”Kendt”, ”Konflikt og krig”, ”Kriminalitet”, ”Kultur”, ”Livsstil”, ”Politik”, ”Samfund”, ”Sport”, ”Sundhed”, ”Teknologi”, ”Transportmiddel”, ”Uddannelse”, ”Underholdning”, ”Vejr”, ”Videnskab”, ”Økonomi”
Giv et præcist svar i json: {{topics: [”kategori”, ”kategori”, ”kategori”, ... ]}}.
"""

# format data
messages = []
for id, text in sampled_data.text.items():
    label = {"topics": data.label[id]}
    label = json.dumps(label)
    article = {
    "messages": [
            {"role": "system", "content": f"{topic_v3_prompt}"},
            {"role": "user", "content": f"Artikel: {text} \nArtiklen omhandler dette/disse emne(r):"},
            {"role": "assistant", "content": label}
            ],
    }

    messages.append(article)


In [ ]:
# save to json
with open("", "w", encoding="utf-8") as json_file:
    for el in messages:
        # Use json.dump to write the list to the file
        json_file.write(json.dumps(el, ensure_ascii=False))
        json_file.write("\n")

## Inference with fine-tuned model

In [ ]:
# Load evaluation data
evaluation_data = pd.read_csv("")

In [ ]:
# prompts
topic_v3_prompt = """
Du er en gennemsnitlig dansk nyhedsforbruger. Du får en artikel, og skal tildele den et passende antal kategorier svarende til de emner den omhandler. Artiklen omhandler oftest flere emner, men en kan også kun omhandle ét emne. 
Her er nogle generelle principper, du skal følge: Du skal anvende generel verdensviden og artiklens kontekst, hvor det er nødvendigt. Et emne skal bidrage med at fremføre artiklens væsentligste pointer for at kategorien tildeles. Det er muligt at samtagge flere kategorier, så to kategorier tilsammen skaber det emne, artiklen omhandler.
Kategorier: ”Begivenhed”: En hændelse, der finder sted - særligt for at markere noget vigtigt og evt. tilbagevendende. Herunder mærkedag, personlig begivenhed, sportsbegivenhed og underholdningsbegivenhed. ”Bolig”: Privatpersoners hjem. Herunder køb og salg, renovering/indretning og udlejning. ”Erhverv”: Emner relateret til beskæftigelsesmæssige forhold, inkl. arbejdspladser. Herunder ansættelsesforhold, offentlig instans og privat virksomhed. ”Dyr”: Levende flercellede organismer, ekskl. planter og mennesker. ”Katastrofe”: Begivenheder, som har negative konsekvenser for en større eller mindre gruppe (mennesker). Herunder mindre ulykke og større katastrofe. ”Kendt”: Personer, som ofte er alment kendt blandt den brede befolkning, fx deltagere i tv-programmer, politikere, sportspersonligheder, toperhvervsfolk og kongelige. ”Konflikt og krig”: Sammenstød, uoverensstemmelser eller stridigheder mellem to eller flere parter - evt. med voldelige konsekvenser. Herunder terror og væbnet konflit. ”Kriminalitet”: Ulovligheder, der omhandler fx dramatiske, dødelige eller sindsoprivende begivenheder, hvori politiet evt. har været involveret. Herunder bandekriminalitet, bedrageri og personfarlig kriminalitet. ”Kultur”: Den levevis, som er resultat af menneskelig aktivitet, og forestillingsverden, herunder skikke, holdninger og traditioner, ved en bestemt befolkningsgruppe i en bestemt periode. Herunder byliv, kunst, museum og seværdighed, or rejse. ”Livsstil”: Den bevidst valgte levemåde, som (en) privatperson(er) har - ofte for at signalere en bestemt identitet. Herunder erotik, familieliv, fritid, krop og velvære, mad og drikke og partnerskab. ”Politik”: Samfundsrelevante problemstillinger, værdier o.lign., som udføres på baggrund af en bestemt ideologi. Herunder international politik og national politik. ”Samfund”: Forhold for mennesker, der typisk deler samme geografiske område og/eller er knyttet sammen på anden vis, fx gennem indbyrdes afhængighed og tilknytning til nationalstat. Herunder bæredygtighed og klima, værdier, religion og tendens. ”Sport”: (Fysisk) aktivitet, som udøves individuelt eller på hold i professionelle sammenhænge, og hvor der ofte indgår specifikke regler og evt. udstyr. Herunder cykling, håndbold, fodbold, ketcher- og batsport og motorsport. ”Sundhed”: Helbredsmæssige forhold. Herunder kosmetisk behandling og sygdom og behandling. Teknologi”: Beskrivelser og anvendelser af tekniske hjælpemidler til udførelse af opgaver. Herunder forbrugerelektronik, kunstig intelligens og software. ”Transportmiddel”: Førbare anordninger, som kan fragte levende væsener og/eller artefakter. Herunder bil, mindre transportmiddel, offentlig transport og større transportmiddel. ”Uddannelse”: Strukturerede undervisningsforløb. Herunder grundskole, ungdomsuddannelse og videregående uddannelse. "Underholdning”: Forhold med evne til at more/frembringe glæde hos modtageren. Herunder litteratur, film og tv, musik og lyd og reality. ”Vejr”: Meterologiske forhold, fx forudsigelse eller afrapportering, ofte omhandlende et bestemt geografisk område på et bestemt tidspunkt. "Videnskab”: Metodologisk forskning og undersøgelse af bestemte dele af virkeligheden. Herunder naturvidenskab og samfundsvidenskab og humaniora. ”Økonomi”: Produktion, fordeling og forbrug (hos både privatpersoner og på statsligt niveau) af varer og tjenester. Herunder makroøkonomi og mikroøkonomi. 
Giv et præcist svar i json: {{topics: [”kategori”, ”kategori”, ”kategori”, ... ]}}.
"""

topic_v0_prompt = """
Du er en gennemsnitlig dansk nyhedsforbruger. Du får en artikel, og skal tildele den et passende antal kategorier svarende til de emner den omhandler. Artiklen omhandler oftest flere emner, men den kan også omhandle kun ét emne.
Kategorier: ”Begivenhed”, ”Bolig”, ”Erhverv”, ”Dyr”, ”Katastrofe”, ”Kendt”, ”Konflikt og krig”, ”Kriminalitet”, ”Kultur”, ”Livsstil”, ”Politik”, ”Samfund”, ”Sport”, ”Sundhed”, ”Teknologi”, ”Transportmiddel”, ”Uddannelse”, ”Underholdning”, ”Vejr”, ”Videnskab”, ”Økonomi”
Giv et præcist svar i json: {{topics: [”kategori”, ”kategori”, ”kategori”, ... ]}}.
"""

In [ ]:
def zeroshot_topic_annotation(text, prompt):
    try:
        response = client.chat.completions.create(
            model="", # specify fine-tuned model
            messages=[
                {
                    "role": "system", 
                    "content": f"{prompt}"
                },
                {
                    "role": "user", 
                    "content": f"Artikel: {text} \nArtiklen omhandler dette/disse emne(r):"
                }
            ],
            temperature=0,
            response_format={
                "type": "json_schema",
                "json_schema": {
                    "name": "topic_schema",
                    "schema": {
                        "type": "object",
                        "properties": {
                            "topics": {
                                "description": "Topics of the article",
                                "type": "string"
                            },
                            "additionalProperties": False
                        }
                    }
                }
            }
        )

        # Extract  response
        topics = response.choices[0].message.content
        return topics
    except Exception as e:
        print(f"Error: {e}")
        return None


def fewshot_topic_annotation(text, prompt, fewshot_dataset):
    # Sample few-shot examples from dataset without current article
    fewshot_dataset = fewshot_dataset[fewshot_dataset['text'] != text]
    fewshot_examples = fewshot_dataset.sample(3).to_dict(orient='records')
    # Create few-shot parrt of prompt
    fewshot_examples = "\n".join([f"Artikel: {example['text']}. Artiklen omhandler dette/disse emne(r): {example['label']}." for example in fewshot_examples])
    try:
        response = client.chat.completions.create(
            model="", # specify fine-tuned model
            messages=[
                {
                    "role": "system", 
                    "content": f"{prompt}. Her er tre eksempler på artikler og deres emner: {fewshot_examples}"
                },
                {
                    "role": "user", 
                    "content": f"Artikel: {text} \nArtiklen omhandler dette/disse emne(r):"
                }
            ],
            temperature=0,
            response_format={
                "type": "json_schema",
                "json_schema": {
                    "name": "topic_schema",
                    "schema": {
                        "type": "object",
                        "properties": {
                            "topics": {
                                "description": "Topics of the article",
                                "type": "string"
                            },
                            "additionalProperties": False
                        }
                    }
                }
            }
        )

        # Extract  response
        topics = response.choices[0].message.content
        return topics
    except Exception as e:
        print(f"Error: {e}")
        return None

In [ ]:
# Apply function and add results to df
evaluation_data["llm_annotation"] = evaluation_data["text"].apply(lambda text: zeroshot_topic_annotation(text, prompt=topic_v3_prompt))

# Save the results to a new CSV file
evaluation_data.to_csv("", index=False)